In [1]:
import dash
import dash_core_components as dcc
import plotly.graph_objs as go
import dash_html_components as html
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output
import dash_table

In [2]:
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
us_state_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_state.csv')
us_death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
us_confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

In [3]:
# data cleaning

# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
us_state_df.columns = map(str.lower, us_state_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
us_confirmed_df.columns = map(str.lower, us_confirmed_df.columns)
us_death_df.columns = map(str.lower, us_death_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = recovered_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})
us_confirmed_df = us_confirmed_df.rename(columns={'province_state': 'state', 'country_region': 'country'})
us_death_df = us_death_df.rename(columns={'province_state': 'state', 'country_region': 'country'})
us_state_df = us_state_df.rename(columns={'country_region': 'country','province_state' : 'state'})
us_state_df = us_state_df[us_state_df.country =='US']
# us_state_df.fillna(0, inplace=True)

In [4]:
death_df.head()

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,30,30,30,33,36,36,40,42,43,47
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,26,26,26,26,26,26,27,27,27,27
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,348,364,367,375,384,392,402,407,415,419
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,33,35,35,36,37,37,37,37,40,40
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [5]:
recovered_df.head()

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,54,99,112,131,135,150,166,179,188,188
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,277,283,302,314,327,345,356,385,394,403
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,783,846,894,1047,1099,1152,1204,1355,1408,1479
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,169,191,205,235,248,282,309,333,344,344
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,5,5,6,6,6,6,6,6,6,6


In [6]:
country_df.head()

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
0,Australia,2020-04-26 21:31:19,-25.0000,133.0000,6694,80,5364,1250,26.292533,NaN,NaN,1.195100,36,AUS
1,Austria,2020-04-26 21:30:32,47.5162,14.5501,15225,542,12282,2401,169.046456,NaN,NaN,3.559934,40,AUT
2,Canada,2020-04-26 21:30:54,60.0010,-95.0010,46866,2652,16805,27409,123.801693,NaN,NaN,5.658686,124,CAN
3,China,2020-04-26 13:43:35,30.5928,114.3055,83909,4636,78188,1085,5.973547,NaN,NaN,5.525033,156,CHN
4,Denmark,2020-04-26 21:30:32,56.0000,10.0000,8773,422,5994,2357,151.462233,NaN,NaN,4.810213,208,DNK


In [7]:
us_confirmed_df.head()

,uid,iso2,iso3,code3,fips,admin2,state,country,lat,long_,...,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,135,136,136,136,136,136,136,139,141,141
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,13,13,14,14,14,14,14,14,14,14
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,1043,1068,1118,1213,1252,1298,1252,1416,1276,1307
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,51,51,53,53,53,53,54,54,54,55


In [8]:
us_death_df.head()
us_death_df.drop(columns=['population'])

,uid,iso2,iso3,code3,fips,admin2,state,country,lat,long_,...,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,...,0,0,0,0,0,0,0,0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,...,5,5,5,5,5,5,5,5,5,5
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,...,2,2,2,2,2,2,2,2,2,2
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.220800,-66.590100,...,56,58,60,62,63,64,63,69,77,83
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.335800,-64.896300,...,1,2,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3257,84070017.0,US,USA,840,NaN,Southeast Utah,Utah,US,38.996171,-110.701396,...,0,0,0,0,0,0,0,0,0,0
3258,84070018.0,US,USA,840,NaN,Southwest Utah,Utah,US,37.854472,-111.441876,...,0,0,0,1,1,1,1,1,1,2
3259,84070019.0,US,USA,840,NaN,TriCounty,Utah,US,40.124915,-109.517442,...,0,0,0,0,0,0,0,0,0,0
3260,84070020.0,US,USA,840,NaN,Weber-Morgan,Utah,US,41.271160,-111.914512,...,0,0,0,1,1,2,2,2,2,2


In [9]:
us_state_df.head(15)

,state,country,last_update,lat,long_,confirmed,deaths,recovered,active,fips,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3,testing_rate,hospitalization_rate
0,Alabama,US,2020-04-26 21:32:45,32.3182,-86.9023,6270,216,NaN,6054.0,1.0,133.720148,73551.0,845.0,3.444976,84000001,USA,1568.620509,13.476874
1,Alaska,US,2020-04-26 21:32:45,61.3707,-152.4044,340,9,217.0,331.0,2.0,56.880062,16177.0,14.0,2.647059,84000002,USA,2706.319877,4.117647
2,American Samoa,US,2020-04-26 21:32:45,-14.2710,-170.1320,0,0,NaN,NaN,60.0,0.000000,3.0,NaN,NaN,16,ASM,5.391708,NaN
3,Arizona,US,2020-04-26 21:32:45,33.7298,-111.4312,6534,275,1383.0,6259.0,4.0,89.768568,64811.0,1037.0,4.208754,84000004,USA,890.417913,15.870830
4,Arkansas,US,2020-04-26 21:32:45,34.9697,-92.3731,2941,49,985.0,2892.0,5.0,113.594798,39469.0,291.0,1.666100,84000005,USA,1524.472321,9.894594
5,California,US,2020-04-26 21:32:45,36.1162,-119.6816,42952,1706,NaN,41246.0,6.0,109.547339,526084.0,4928.0,3.971876,84000006,USA,1341.755971,11.473272
6,Colorado,US,2020-04-26 21:32:45,39.0598,-105.3111,12968,684,2150.0,12284.0,8.0,228.841149,59791.0,2410.0,5.274522,84000008,USA,1055.108048,18.584207
7,Connecticut,US,2020-04-26 21:32:45,41.5978,-72.7554,25269,1930,NaN,23339.0,9.0,708.750796,79811.0,1766.0,7.637817,84000009,USA,2238.557513,6.988801
8,Delaware,US,2020-04-26 21:32:45,39.3185,-75.5071,4034,120,911.0,3914.0,10.0,414.268755,19249.0,317.0,2.974715,84000010,USA,1976.762337,7.858205
9,Diamond Princess,US,2020-04-26 21:32:45,NaN,NaN,49,0,0.0,49.0,88888.0,NaN,NaN,NaN,0.000000,84088888,USA,NaN,NaN


In [10]:
sorted_us_state_df = us_state_df.sort_values('confirmed', ascending= False)
sorted_us_state_df.head()

,state,country,last_update,lat,long_,confirmed,deaths,recovered,active,fips,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3,testing_rate,hospitalization_rate
36,New York,US,2020-04-26 21:32:45,42.1657,-74.9481,288045,22246,23887.0,265799.0,36.0,1708.348099,805350.0,57103.0,7.723099,84000036,USA,4776.400011,19.824333
34,New Jersey,US,2020-04-26 21:32:45,40.2989,-74.5210,109038,5938,NaN,103100.0,34.0,1227.602652,223144.0,6573.0,5.445808,84000034,USA,2512.263304,6.028174
25,Massachusetts,US,2020-04-26 21:32:45,42.2302,-71.5301,54938,2730,NaN,52208.0,25.0,800.405375,236100.0,5104.0,4.969238,84000025,USA,3439.799574,9.290473
17,Illinois,US,2020-04-26 21:32:45,40.3495,-88.9861,43903,1934,NaN,41969.0,17.0,374.099336,214952.0,4595.0,4.405166,84000017,USA,1831.615164,10.466255
5,California,US,2020-04-26 21:32:45,36.1162,-119.6816,42952,1706,NaN,41246.0,6.0,109.547339,526084.0,4928.0,3.971876,84000006,USA,1341.755971,11.473272


In [11]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)
sorted_country_df.head()

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
17,US,2020-04-26 21:31:04,40.000000,-100.00000,963168,54614,106518,802036,292.341903,NaN,NaN,5.670247,840,USA
160,Spain,2020-04-26 21:30:32,40.463667,-3.74922,226629,23190,117727,85712,484.718323,NaN,NaN,10.232583,724,ESP
10,Italy,2020-04-26 21:30:32,41.871900,12.56740,197675,26644,64928,106103,326.941819,NaN,NaN,13.478690,380,ITA
6,France,2020-04-26 21:30:32,46.227600,2.21370,161665,22890,45681,93094,247.673206,NaN,NaN,14.158909,250,FRA
7,Germany,2020-04-26 21:30:32,51.165700,10.45150,157495,5944,112000,39551,187.977542,NaN,NaN,3.774088,276,DEU


In [12]:
table_columns = sorted_country_df[['country','last_update','confirmed','recovered','deaths','active','incident_rate','mortality_rate']]
us_table_columns = sorted_us_state_df[['state','country','last_update','confirmed','recovered','deaths','active','incident_rate','mortality_rate']]

In [13]:
colors = {
    'text' : '#AF2406',
    'plot_color' : '#D3D3D3',
    'paper_color' : 'CFE9E8'
}

In [14]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

In [15]:
# total number of confirmed, death and recovered cases
us_confirmed_total = int(us_state_df['confirmed'].sum())
us_deaths_total = int(us_state_df['deaths'].sum())
us_active_total = int(us_state_df['active'].sum())

In [80]:
app = dash.Dash()

In [81]:
app.layout = html.Div([
    html.H1(children = "COVID-19 CORONAVIRUS PANDEMIC",
            style = {
                'textAlign' : 'center',
                'color' : colors['text']
           }),
    html.H3(children = 'Coronavirus spread summary and tracker - World',
            style = {
                'textAlign' : 'center',
                'color' : colors['text']
           }),
    html.Img(src = "https://www.stjosephshealth.org/images/covid19-header.jpg",
            style = {
                'width' : '100%',
                'height' : '400px',
                'border': 'thin grey solid'
            }),
    html.Br(),
    html.Br(),
    html.Div([
        html.Span(children = 'Confirmed Cases: '+ str(confirmed_total),
              style = {
                  'color' : 'blue',
                  'font-size' : '30px'
           }),
        html.Span(children = 'Active Cases: '+ str(active_total),
             style = {
                  'color' : 'orange',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }),
        html.Span(children = 'Deaths: '+ str(deaths_total),
              style = {
                  'color' : 'red',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }),
        html.Span(children = 'Recovered: '+ str(recovered_total),
              style = {
                  'color' : 'green',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }),
        ],
    style = {
          'textAlign' : 'center',
    }),
    html.Br(),
    html.Br(),
    html.Div([
         html.Label(children ="number of countries: ",
                   style = {
                       'color' : 'blue',
                      'font-size' : '20px'
                   }),
        dcc.Slider(
            id = 'country-slider',
            min = 1,
            max = 50,
            value = 10,
            step = 1,
            marks = {i : i for i in range(51)}
        )],
#         style ={
#             'width' : '100%'
#         }),
        ),
    html.Br(),
    html.Br(),
    dash_table.DataTable(
        id = 'table-world-cases',
        columns=[{"name": i, "id": i} for i in table_columns],
        #data=sorted_country_df.to_dict('records'),
        style_table={'overflowX': 'scroll',
                    'maxHeight': '425px',
                    'overflowY': 'scroll'},
        fixed_rows={ 'headers': True, 'data': 0 },
         style_cell={
        'height': 'auto',
        'minWidth': '120px', 'maxWidth': '140px',
        'whiteSpace': 'normal',
        'textAlign': 'left',
         'backgroundColor': 'rgb(250, 250, 250)',
        },
        style_header={
        'height' : '40px',
        'backgroundColor': 'rgb(230, 230, 230)',
        'fontWeight': 'bold'
        },
        style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        },
        {
            'if': {'column_id': 'country'},
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'confirmed'},
            'backgroundColor': '#0AD3F7',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'deaths'},
            'backgroundColor': '#AF2406',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'recovered'},
            'backgroundColor': '#3D9970',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'active'},
            'backgroundColor': '#FC7E27',
            'fontWeight': 'bold'     
        }
        ],
    ),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-world-cases',
            figure = {
                'data' : [
                    go.Bar(
                        name = 'Confirmed Cases',
                        x = np.array(list(confirmed_df.iloc[:, 20:].columns)),
                        y = np.sum(np.asarray(confirmed_df.iloc[:,4:]),axis = 0),
                    ),
                    go.Bar(
                       name = 'Deaths', 
                       x = np.array(list(death_df.iloc[:, 20:].columns)),
                       y = np.sum(np.asarray(death_df.iloc[:,4:]),axis = 0),
                    ),
                    go.Bar(
                       name = 'Recoveries',
                       x = np.array(list(recovered_df.iloc[:, 20:].columns)),
                       y = np.sum(np.asarray(recovered_df.iloc[:,4:]),axis = 0),
                    )
                ],
                'layout' : go.Layout(
                    title = "Cases around the world",
                    xaxis = {'title' : 'Countries'},
                    yaxis = {'title' : 'Cases'},
                    hovermode = 'closest',
                    height=400,
                    barmode = 'stack',
                    showlegend = True
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        #'padding' : '30px'
        }),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Dropdown(
            id = 'case-type',
            options = [
                {'label' : "Confirmed Cases", 'value' : 'confirmed'},
                {'label' : "Active Cases", 'value' : 'active'},
                {'label' : "Death Cases", 'value' : 'deaths'},
                {'label' : "Recovered Cases", 'value' : 'recovered'}
            ],
            value = 'confirmed',
        )],
         style ={
            'width' : '50%'
        }),
    html.Br(),
    html.Br(),
    html.Div([ 
        dcc.Graph(
            id = 'piechart-world-cases')],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
#         'padding' : '30px'
        }),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-world-confirmed-cases',
            figure = {
                'data' : [
                    go.Scatter(
                        name = 'country',
                        x=list(sorted_country_df.country),
                        y=list(sorted_country_df.confirmed),
                        mode='markers',
                        #marker = dict(color = '#0AD3F7')
                        marker = dict(
                            size =  sorted_country_df.confirmed, 
                            sizemin = 4,
                            sizemode='area',
                            sizeref = 2.*max(sorted_country_df.confirmed)/(100**2),
                            color=np.random.randn(300),
                            colorscale='Rainbow',),
                    )],
                'layout' : go.Layout(
                    title = "Confirmed cases around the world",
                    xaxis = {'title' : 'Countries',
                            'automargin' : True
                            },
                    yaxis = {'title' : 'Confirmed Cases',
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest'
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        #'padding' : '30px'
        }),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-world-death-cases',
            figure = {
                'data' : [
                    go.Scatter(
                        name = 'country',
                        x=list(sorted_country_df.country),
                        y=list(sorted_country_df.deaths),
                        mode='lines',
                        marker = dict(
                            color = 'red')
                    )],
                'layout' : go.Layout(
                    title = "Deaths around the world",
                    xaxis = {'title' : 'Countries',
                            'automargin' : True},
                    yaxis = {'title' : 'Deaths',
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest'
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        #'padding' : '30px'
        }),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-world-recovery-cases',
            figure = {
                'data' : [
                    go.Scatter(
                        name = 'country',
                        x=list(sorted_country_df.country),
                        y=list(sorted_country_df.recovered),
                        mode='markers+lines',
                        marker = dict(
                            color = 'green')
                    )],
                'layout' : go.Layout(
                    title = "Recoveries around the world",
                    xaxis = {'title' : 'Countries',
                            'automargin' : True},
                    yaxis = {'title' : 'Recoveries',
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest'
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        #'padding' : '30px'
        }),
    html.Br(),
    html.Br(),
    html.Div([
        html.Label(children ="Country (type a country name): ",
                   style = {
                       'color' : 'blue',
                      'font-size' : '20px'
                   }),
        dcc.Input(
            id = 'country-input',
            #placeholder = 'Input Country Name',
            type = 'text',
            value = 'World'
        )],
        style ={
            'width' : '50%'
        }),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(id = 'Check-country-cases')],
    style = {
        'border' :'solid 1px green',
    }),
    html.Br(),
    html.Br(),
    html.Div([
       dcc.Graph(
            id = 'world-geo-case-tracking',
            figure = {
                'data' : [
                    go.Choropleth(
                        locations=country_df['country'],
                        z=country_df['confirmed'].astype('int64'),
                        locationmode='country names',
                        colorscale='Reds',
                        autocolorscale=False,
                         text = country_df['country']  + '<br>' + 'Confirmed: ' + 
                        country_df['confirmed'].astype(str)  + '<br>' +'Active: ' + 
                        country_df['active'].astype(str)  + '<br>' + 'Deaths: ' + 
                        country_df['deaths'].astype(str)  + '<br>' + 'Recovered: ' + country_df['recovered'].astype(str),
                        marker_line_color='black', # line markers between states
                        colorbar_title="COVID19 confirmed cases",
                    )],
                'layout' : go.Layout(
                    title_text = "Spread of Coronavirus across the World",
                    height=500,
                    hovermode = 'closest',
                    geo = dict(
                        scope='world',
                        #projection=go.layout.geo.Projection(type = 'albers usa'),
                        showlakes=True, # lakes
                        lakecolor='rgb(255, 255, 255)'
                    ),
                )})],
    style = {
        'border' :'solid 1px green',
        #'padding' : '10px'
    }
    ),
    html.Br(),
     html.H3(children = 'Coronavirus spread summary and tracker - US',
            style = {
                'textAlign' : 'center',
                'color' : colors['text']
           }),
    html.Div([
        html.Span(children = 'US Confirmed Cases: '+ str(us_confirmed_total),
              style = {
                  'color' : 'blue',
                  'font-size' : '30px'
           }),
        html.Span(children = 'US Active Cases: '+ str(us_active_total),
             style = {
                  'color' : 'orange',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }),
        html.Span(children = 'US Deaths: '+ str(us_deaths_total),
              style = {
                  'color' : 'red',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }),
        ],
    style = {
          'textAlign' : 'center',
    }),
    html.Br(),
    html.Br(),
    html.Div([
         html.Label(children ="number of states: ",
                   style = {
                       'color' : 'blue',
                      'font-size' : '20px'
                   }),
        dcc.Slider(
            id = 'state-slider',
            min = 1,
            max = 50,
            value = 10,
            step = 1,
            marks = {i : i for i in range(51)}
        )],
#         style ={
#             'width' : '100%'
#         }),
        ),
    html.Br(),
    html.Br(),
    dash_table.DataTable(
        id = 'table-US-cases',
        columns=[{"name": i, "id": i} for i in us_table_columns],
        #data=sorted_country_df.to_dict('records'),
        style_table={'overflowX': 'scroll',
                    'maxHeight': '425px',
                    'overflowY': 'scroll'},
        fixed_rows={ 'headers': True, 'data': 0 },
         style_cell={
        'height': 'auto',
        'minWidth': '120px', 'maxWidth': '140px',
        'whiteSpace': 'normal',
        'textAlign': 'left',
         'backgroundColor': 'rgb(250, 250, 250)',
        },
        style_header={
        'height' : '40px',
        'backgroundColor': 'rgb(230, 230, 230)',
        'fontWeight': 'bold'
        },
        style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        },
        {
            'if': {'column_id': 'country'},
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'state'},
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'confirmed'},
            'color': '#0B53E6',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'deaths'},
            'color': '#AF2406',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'recovered'},
            'color': '#3D9970',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'active'},
            'color': '#FC7E27',
            'fontWeight': 'bold'     
        }
        ],
    ),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-us-cases',
            figure = {
                'data' : [
                    go.Bar(
                        name = 'Confirmed Cases',
                        x = np.array(list(us_confirmed_df.iloc[:, 21:].columns)),
                        y = np.sum(np.asarray(us_confirmed_df.iloc[:,12:]),axis = 0),
                    ),
                    go.Bar(
                       name = 'Deaths', 
                       x = np.array(list(us_death_df.iloc[:, 25:].columns)),
                       y = np.sum(np.asarray(us_death_df.iloc[:,12:]),axis = 0),
                    ),
                ],
                'layout' : go.Layout(
                    title = "Cases around the US",
                    xaxis = {'title' : 'States'},
                    yaxis = {'title' : 'Cases'},
                    hovermode = 'closest',
                    height=400,
                    barmode = 'stack',
                    showlegend = True
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        #'padding' : '30px'
        }),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Dropdown(
            id = 'us-case-type',
            options = [
                {'label' : "Confirmed Cases", 'value' : 'confirmed'},
                {'label' : "Active Cases", 'value' : 'active'},
                {'label' : "Death Cases", 'value' : 'deaths'},
            ],
            value = 'confirmed',
        )],
         style ={
            'width' : '50%'
        }),
    html.Br(),
    html.Br(),
    html.Div([ 
        dcc.Graph(
            id = 'piechart-US-cases')],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
#         'padding' : '30px'
        }),
        html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-us-confirmed-cases',
            figure = {
                'data' : [
                    go.Scatter(
                        name = 'state',
                        x=list(sorted_us_state_df.state),
                        y=list(sorted_us_state_df.confirmed),
                        mode='markers',
                        marker = dict(
                            size =  sorted_country_df.confirmed, 
                            sizemin = 4,
                            sizemode='area',
                            sizeref = 2.*max(sorted_country_df.confirmed)/(100**2),
                            color=np.random.randn(300),
                            symbol = 'square',
                            colorscale='Rainbow',),
                    )],
                'layout' : go.Layout(
                    title = "Confirmed cases around US",
                    xaxis = {'title' : 'States',
                            'automargin' : True
                            },
                    yaxis = {'title' : 'Confirmed Cases',
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest'
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        }),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-us-death-cases',
            figure = {
                'data' : [
                    go.Scatter(
                        name = 'state',
                        x=list(sorted_us_state_df.state),
                        y=list(sorted_us_state_df.deaths),
                        mode='markers+lines',
                        marker = dict(
                            color = 'red')
                    )],
                'layout' : go.Layout(
                    title = "Deaths around US",
                    xaxis = {'title' : 'States',
                            'automargin' : True},
                    yaxis = {'title' : 'Deaths',
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest'
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        #'padding' : '30px'
        }),
    html.Br(),
    html.Br(),
    html.Div([
       dcc.Graph(
            id = 'usa-geo-case-tracking',
            figure = {
                'data' : [
                    go.Scattergeo(
                        lat = us_state_df['lat'],
                        lon = us_state_df['long_'],
                        text = us_state_df['state']  + '<br>' + 'Confirmed: ' + 
                        us_state_df['confirmed'].astype(str)  + '<br>' +'Active: ' + 
                        us_state_df['active'].astype(str)  + '<br>' + 'Deaths: ' + 
                        us_state_df['deaths'].astype(str)  + '<br>' + 'Recovered: ' + us_state_df['recovered'].astype(str),
                        mode = 'markers',
                        marker = dict(
                            size =  us_state_df.confirmed/5, 
                            sizemin = 4,
                            sizemode='area',
                            sizeref = 2.*max(us_state_df.confirmed)/(100**2),
                            color = 'rgba(135, 206, 250, 0.8)',
                            line = dict(
                            color = 'rgb(250,0,0)',
                              width = 2
                            )
                       ),
                        hoverinfo = 'text',
                        name = 'state'
                    )],
                'layout' : go.Layout(
                    title_text = "Spread of Coronavirus across the US",
                    height=500,
                    hovermode = 'closest',
                    geo = dict(
                        scope = 'usa',
                        showcoastlines=True, coastlinecolor="RebeccaPurple",
                        showland=True, landcolor="LightGrey",
                        projection = dict(type = 'albers usa'),
                    )
                )})],
    style = {
        'border' :'solid 1px green',
        #'padding' : '10px'
    }
    ),
],
style = {
    #'background-color' : '#BBCDF2'
    'padding' : '20px'
})
    


In [82]:
@app.callback(
    Output('table-world-cases', 'data'),
    [Input('country-slider', 'value')])
def display_country_cases_table(number):
    return sorted_country_df.head(number).to_dict('records')


In [83]:
@app.callback(
    Output('table-US-cases', 'data'),
    [Input('state-slider', 'value')])
def display_country_cases_table(number):
    return sorted_us_state_df.head(number).to_dict('records')

In [84]:
@app.callback(
    Output('piechart-world-cases', 'figure'),
    [Input('case-type', 'value')])
def plot_world_info(caseType):
    if(caseType == 'confirmed'):
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_country_df.country,
            values = sorted_country_df.confirmed,
            hole = 0.6,
            hoverinfo = 'label+value+percent',
            textposition = 'inside'
        )
    elif(caseType == 'active'):
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_country_df.country,
            values = sorted_country_df.active,
            hole = 0.6,
            hoverinfo = 'label+value+percent',
            textposition = 'inside'
        )
    elif(caseType == 'deaths'):
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_country_df.country,
            values = sorted_country_df.deaths,
            hole = 0.6,
            hoverinfo = 'label+value+percent',
            textposition = 'inside'
        )
    else:
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_country_df.country,
            values = sorted_country_df.recovered,
            hole = 0.6,
            hoverinfo = 'label+value+percent',
            textposition = 'inside'
        )
    return {
        'data': [trace],
        'layout': dict(
            title = str(caseType) + ' cases around the world',
            showlegend = True,
            hovermode = 'closest',
            uniformtext_minsize=8,
            uniformtext_mode='hide',
        )
    }
    

In [85]:
@app.callback(
    Output('piechart-US-cases', 'figure'),
    [Input('us-case-type', 'value')])
def plot_world_info(caseType):
    if(caseType == 'confirmed'):
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_us_state_df.state,
            values = sorted_us_state_df.confirmed,
            hole = 0.6,
            hoverinfo = 'label+value+percent',
            textposition = 'inside'
        )
    elif(caseType == 'active'):
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_us_state_df.state,
            values = sorted_us_state_df.active,
            hole = 0.6,
            hoverinfo = 'label+value+percent',
            textposition = 'inside'
        )
    elif(caseType == 'deaths'):
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_us_state_df.state,
            values = sorted_us_state_df.deaths,
            hole = 0.6,
            hoverinfo = 'label+value+percent',
            textposition = 'inside'
        )
    return {
        'data': [trace],
        'layout': dict(
            title = str(caseType) + ' cases around US',
            showlegend = True,
            hovermode = 'closest',
            uniformtext_minsize=8,
            uniformtext_mode='hide',
        )
    }
    

In [86]:
@app.callback(
    Output('Check-country-cases', 'figure'),
    [Input('country-input', 'value')])
def plot_country_info(country):
    labels = ['confirmed','deaths','recovered']
    colors = ['orange','red','green']
    mode_size = [7, 7, 7]
    line_size = [4, 4, 4]
    
    df_list = [confirmed_df, death_df, recovered_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country.lower() == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'].str.lower() == country.lower()].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        height = 500,        
    );
    return fig

In [87]:
if __name__ == '__main__':
    app.run_server(port = 4048,debug=False,use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4048/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Apr/2020 18:15:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2020 18:15:34] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2020 18:15:34] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2020 18:15:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2020 18:15:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2020 18:15:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2020 18:15:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2020 18:15:34] "POST /_dash-update-component HTTP/1.1" 200 -
